In [1]:
import os
import warnings
import sys
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [2]:
pj_dir = Path(os.getcwd()).parent

In [3]:
mlflow.set_tracking_uri(str(pj_dir/'mlruns'))

In [4]:
mlflow.tracking.get_tracking_uri()

'/Users/yutaro_ikeda/.ghq/github.com/ikedaosushi/python-sandbox/mlflow/mlruns'

In [5]:
# mlflow.create_experiment(name='tutorials')

In [6]:
from sklearn import datasets

In [7]:
data, target = datasets.load_boston(return_X_y=True)
train_x, test_x, train_y, test_y = train_test_split(data, target)

In [8]:
alpha = 0.7
l1_ratio = 0.7

with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.700000, l1_ratio=0.700000):
  RMSE: 5.914940849183863
  MAE: 4.103179824615929
  R2: 0.6171503032831637


In [10]:
mlflow.search_runs()

,run_id,experiment_id,status,artifact_uri,start_time,end_time
